In [15]:
# Importing the Psi4 module
import psi4


In [16]:
%%time
# Setting up the output file
psi4.set_output_file('output.dat', False)


CPU times: user 0 ns, sys: 776 µs, total: 776 µs
Wall time: 1.03 ms


PosixPath('output.dat')

In [17]:
%%time
# Setting memory for Psi4
psi4.set_memory('500 MB')

# Defining the geometry of the water molecule
h2o = psi4.geometry("""
O
H 1 0.96
H 1 0.96 2 104.5
""")

# Running a Hartree-Fock calculation with a cc-pVDZ basis set
psi4.energy('scf/cc-pvdz')


CPU times: user 296 ms, sys: 11 ms, total: 307 ms
Wall time: 329 ms


-76.0266327350902

In [18]:
%%time
# Defining bond length and angle for CH₂
R = 1.075
A = 133.93

# Defining the geometry of the CH₂ molecule with triplet state
ch2 = psi4.geometry("""
0 3
C
H 1 {0}
H 1 {0} 2 {1}
""".format(R, A))

# Setting options for an unrestricted Hartree-Fock calculation
psi4.set_options({'reference': 'uhf'})

# Calculating the energy
psi4.energy('scf/6-31g**')


CPU times: user 288 ms, sys: 9.85 ms, total: 298 ms
Wall time: 301 ms


-38.92533462893707

In [19]:
%%time
# Setting options for restricted Hartree-Fock
psi4.set_options({'reference': 'rhf'})

# Optimizing geometry
psi4.optimize('scf/cc-pvdz', molecule=h2o)


Optimizer: Optimization complete!
CPU times: user 859 ms, sys: 24.9 ms, total: 884 ms
Wall time: 894 ms


-76.02703272937481

In [20]:
%%time

# Calculating vibrational frequencies
scf_e, scf_wfn = psi4.frequency('scf/cc-pvdz', molecule=h2o, return_wfn=True)


CPU times: user 1.02 s, sys: 20.9 ms, total: 1.04 s
Wall time: 1.05 s


In [21]:
%%time
# Defining the geometry of the ethene-ethyne dimer
dimer = psi4.geometry("""
0 1
C   0.000000  -0.667578  -2.124659
C   0.000000   0.667578  -2.124659
H   0.923621  -1.232253  -2.126185
H  -0.923621  -1.232253  -2.126185
H  -0.923621   1.232253  -2.126185
H   0.923621   1.232253  -2.126185
--
0 1
C   0.000000   0.000000   2.900503
C   0.000000   0.000000   1.693240
H   0.000000   0.000000   0.627352
H   0.000000   0.000000   3.963929
units angstrom
""")

# Setting options for SAPT calculation
psi4.set_options({'scf_type': 'df', 'freeze_core': True})

# Running SAPT calculation
psi4.energy('sapt0/jun-cc-pvdz', molecule=dimer)


building SAPT EXCHSCAL1 ....................................SUCCESS
building SAPT EXCHSCAL3 ....................................SUCCESS
building SAPT EXCHSCAL .....................................SUCCESS
building SAPT HF(2) ALPHA=0.0 ENERGY .......................SUCCESS
building SAPT HF(2),U ALPHA=0.0 ENERGY .....................EMPTY, missing SAPT IND20,U ENERGY
building SAPT HF(2) ENERGY .................................SUCCESS
building SAPT HF(2),U ENERGY ...............................EMPTY, missing SAPT HF(2),U ALPHA=0.0 ENERGY
building SAPT HF(3) ENERGY .................................EMPTY, missing SAPT IND30,R ENERGY
building SAPT MP2(2) ENERGY ................................EMPTY, missing SAPT MP2 CORRELATION ENERGY
building SAPT MP2(3) ENERGY ................................EMPTY, missing SAPT MP2(2) ENERGY
building SAPT MP4 DISP .....................................EMPTY, missing SAPT DISP21 ENERGY
building SAPT CCD DISP .....................................EMPTY, missing

-0.0022355823909805178

In [22]:
%%time
# Defining geometry for Neon dimer with variable distance
ne2_geometry = """
Ne
--
Ne 1 {0}
"""

# List of interatomic distances
Rvals = [2.5, 3.0, 4.0]

# Setting options
psi4.set_options({'freeze_core': True})

# Counterpoise correction calculation
ecp = {}
for R in Rvals:
    ne2 = psi4.geometry(ne2_geometry.format(R))
    ecp[R] = psi4.energy('ccsd(t)/aug-cc-pvdz', bsse_type='cp', molecule=ne2)

# Displaying results
print("CP-corrected CCSD(T)/aug-cc-pVDZ Interaction Energies\n\n")
print("          R [Ang]                 E_int [kcal/mol]       ")
print("---------------------------------------------------------")
for R in Rvals:
    e = ecp[R] * psi4.constants.hartree2kcalmol
    print("            {:3.1f}                        {:1.6f}".format(R, e))


CP-corrected CCSD(T)/aug-cc-pVDZ Interaction Energies


          R [Ang]                 E_int [kcal/mol]       
---------------------------------------------------------
            2.5                        0.758605
            3.0                        0.015968
            4.0                        -0.016215
CPU times: user 5.78 s, sys: 1.61 s, total: 7.38 s
Wall time: 11.6 s
